In [1]:
import numpy as np
import pandas as pd 

In [2]:
df=pd.read_csv('Data_for_UCI_named.csv')

In [3]:
df.drop('stab',inplace=True,axis=1)

In [4]:
X=df.drop('stabf',axis=1)
y=df['stabf']

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [46]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [48]:
xgboost=XGBClassifier(random_state=1)
lightgbm=LGBMClassifier(random_state=1)

In [50]:
extra_tree=ExtraTreesClassifier(random_state=1)
random_forest=RandomForestClassifier(random_state=1)

In [35]:
from sklearn.preprocessing import StandardScaler

In [36]:
SCALER=StandardScaler()

In [37]:
X_train_scaled=SCALER.fit_transform(X_train)

In [38]:
X_test_scaled=SCALER.transform(X_test)

In [51]:
random_forest.fit(X_train_scaled,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [52]:
random_forest_pred=extra_tree.predict(X_test_scaled)

NotFittedError: This ExtraTreesClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [53]:
extra_tree.fit(X_train_scaled,y_train)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
                     oob_score=False, random_state=1, verbose=0,
                     warm_start=False)

In [54]:
extar_tree_pred=extra_tree.predict(X_test_scaled)

In [59]:
xgboost.fit(X_train,y_train)
xgboost_pred=xgboost.predict(X_test)

In [60]:
lightgbm.fit(X_train,y_train)
lightgbm_pred=xgboost.predict(X_test)

In [41]:
from sklearn import metrics

In [42]:
print (metrics.accuracy_score(y_test,extra_tree_pred))
print (metrics.accuracy_score(y_test,random_forest_pred))
print (metrics.accuracy_score(y_test,xgboost_pred))
print (metrics.accuracy_score(y_test,lightgbm_pred))

0.928

In [56]:
from sklearn.model_selection import RandomizedSearchCV

In [62]:
n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None] 

hyperparameter_grid = {'n_estimators': n_estimators,
'min_samples_leaf': min_samples_leaf,
'min_samples_split': min_samples_split,
'max_features': max_features}

random_search = RandomizedSearchCV(extra_tree, hyperparameter_grid, cv=5, n_iter=10, n_jobs=-1, verbose=1, random_state=1, scoring='accuracy')
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.7min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=ExtraTreesClassifier(bootstrap=False,
                                                  class_weight=None,
                                                  criterion='gini',
                                                  max_depth=None,
                                                  max_features='auto',
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=10, n_jobs=None,
                                                  oob_score=False,
          

In [64]:
random_search.best_params_

{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 8,
 'max_features': None}

In [65]:
extra_tree_optimized=ExtraTreesClassifier(n_estimators=1000,min_samples_split=2,min_samples_leaf=8,max_features=None)

In [67]:
extra_tree_optimized.fit(X_train,y_train)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features=None, max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=8, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=1000,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [69]:
extra_tree_optimized_pred=extra_tree_optimized.predict(X_test)

In [71]:
print (metrics.classification_report(y_test,extra_tree_optimized_pred,digits=4))

precision    recall  f1-score   support

      stable     0.9199    0.8708    0.8947       712
    unstable     0.9306    0.9581    0.9441      1288

    accuracy                         0.9270      2000
   macro avg     0.9252    0.9144    0.9194      2000
weighted avg     0.9268    0.9270    0.9265      2000

